In [ ]:
import numpy as np
import plotly.graph_objects as go
import hyperspy.api as hs
from plotly.subplots import make_subplots

In [ ]:
path = "../../../Masteroppgave/2022-09-06_EDS-Apreo/exports/"
s = hs.load(path + 'GaAs_10kV.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'As']
s.set_elements(elements)

In [ ]:
s.data[:33] = 0
m = s.isig[320:1030].create_model()
m.fit()
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset
c = np.arange(0, 2048)
x = s.axes_manager[0].axis
Vacc = s.metadata.Acquisition_instrument.SEM.beam_energy
c2 = np.arange(320, 1030)
x2 = s.axes_manager[0].axis[320:1030]
b = m[0].function(x2) * scale

In [ ]:
def kev2ch(kev):
    return (kev - offset) / scale
def ch2kev(ch):
    return ch * scale + offset

In [ ]:
dh = s.data[320:420]

x = np.arange(3, 4, scale)
m, bb = np.polyfit(x, dh, 1)
x = np.arange(3, 8, scale)
y = m * x + bb

start = s.data[220:319]

b = np.concatenate((start,b))
y = np.concatenate((start,y))

In [ ]:
fig = go.Figure() 
fig.update_layout(width=600, height=400, font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=10, r=10, t=10, b=10))
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
fig.add_trace(go.Scatter(x=x, y=y, name='Charging', line=dict(width=3, color=colors[1])))
fig.add_trace(go.Scatter(x=x2, y=b, name='Normal', line=dict(width=3, color=colors[0])))

# legend in upper right corner
fig.update_layout(legend=dict(x=0.99, y=.99, traceorder="reversed", yanchor="top", xanchor="right"))

fig.update_yaxes(type="log")
fig.update_xaxes(range=[3,10.1], title='Energy [keV]')
fig.update_yaxes(range=[2.15,3.7], title='Intensity [counts]')


In [ ]:
# fig.write_image('../figures/pp_duane-hunt.pdf')
# fig.write_image('../../mastersthesis/figures/pp_duane-hunt.pdf')

# Fiori illustration

In [ ]:
path = "../../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
s = hs.load(path + 'GaSb_30kV_50pA.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'Sb']
# s.set_lines()
s.set_elements(elements)
s.data[:33] = 0
m = s.isig[8.:12.].create_model()
m.fit()
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset
c = np.arange(0, 2048)
x = s.axes_manager[0].axis

xm = m.as_signal().axes_manager[0].axis

b1 = m[0].function(xm) * scale*1
b2 = m[0].function(xm) * scale*4

In [ ]:
# plot
fig = make_subplots(rows=1, cols=3, shared_xaxes=False, vertical_spacing=0.08) #, row_heights=[0.2, 0.8])

fig.update_layout(width=800, height=400, font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=10, r=10, t=10, b=90))
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
fig.add_trace(go.Scatter(x=xm, y=b1, name='Background', line=dict(width=3, color=colors[0]), fill='tozeroy'), row=1, col=1)
fig.add_trace(go.Scatter(x=xm, y=m[1].function(xm) * scale + b1, name='Peak', line=dict(width=3, color=colors[2]), fill='tonexty'), row=1, col=1)
fig.add_trace(go.Scatter(x=xm, y=b2, name='Background', line=dict(width=3, color=colors[0]), fill='tozeroy', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=xm, y=m[1].function(xm) * scale + b2, name='Peak', line=dict(width=3, color=colors[2]), fill='tonexty', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=xm, y=b1, name='Background', line=dict(width=3, color=colors[0]), fill='tozeroy', showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=xm, y=m[1].function(xm) * scale*.3 + b1, name='Peak', line=dict(width=3, color=colors[2]), fill='tonexty', showlegend=False), row=1, col=3)
# log y
# fig.update_yaxes(type="log", range=[2.5, 4.3])
fig.update_yaxes(range=[-600, 14e3])

# put legend in upper right corner
fig.update_layout(legend=dict(x=0.99, y=.99, traceorder="reversed", yanchor="top", xanchor="right"))

fig.update_yaxes(title='Intensity [counts]', row=1, col=1)
fig.update_xaxes(range=[8.6,9.9], title='(a)', row=1, col=1)
fig.update_xaxes(range=[8.6,9.9], title='(b)', row=1, col=2)
fig.update_xaxes(range=[8.6,9.9], title='(c)', row=1, col=3)


# put title at bottom with text Energy [keV]
fig.update_layout(title_text="Energy [keV]", title_x=0.53, title_y=0.015, title_font_size=18)

In [ ]:
# fig.write_image('../figures/pp_fiori_snr.pdf')
# fig.write_image('../../mastersthesis/figures/pp_fiori_snr.pdf')

In [ ]:
# peak shift illustration

# plot Ga Ka 
fig = make_subplots(rows=1, cols=2, shared_xaxes=False, vertical_spacing=0.08) #, row_heights=[0.2, 0.8])
fig.update_layout(width=800, height=400, font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=10, r=10, t=10, b=90))

fig.add_trace(go.Scatter(x=x, y=m[1].function(x) * scale, name='Peak', line=dict(width=3, color=colors[2]), showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=m[1].function(x+.045) * scale, name='Ga Kα peak', line=dict(width=3, color=colors[2])), row=1, col=2)

# add a black dashed line from 9.2517 to 9.2517
fig.add_trace(go.Scatter(x=[9.2517, 9.2517], y=[0, 1e4], name='9.2517 keV', mode='lines', line=dict(width=2, color='black', dash='dash')), row=1, col=2)

fig.add_vline(x=9.2517, line_width=2, line_dash="dash", line_color="black")

# put legend in upper right corner
fig.update_layout(legend=dict(x=0.99, y=.99, traceorder="reversed", yanchor="top", xanchor="right"))

fig.update_yaxes(title='Intensity [counts]', row=1, col=1)
fig.update_xaxes(range=[8.99,9.55], title='(a)', row=1, col=1)
fig.update_xaxes(range=[8.99,9.55], title='(b)', row=1, col=2)

fig.update_layout(title_text="Energy [keV]", title_x=0.53, title_y=0.015, title_font_size=18)

In [ ]:
# fig.write_image('../figures/pp_peak_deviation.pdf')
# fig.write_image('../../mastersthesis/figures/pp_peak_deviation.pdf')

# Strays

In [ ]:
path = "../../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
s = hs.load(path + 'GaAs_30kV_25pA.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'As']
# s.set_lines()
s.set_elements(elements)
s.data[:33] = 0
# m = s.isig[8.:12.].create_model()
m = s.create_model()

m.fit()
m.fit_background()
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset
c = np.arange(0, 2048)
x = s.axes_manager[0].axis

xm = m.as_signal().axes_manager[0].axis

b = m[0].function(xm) * scale

In [ ]:
fig = go.Figure() 
fig.update_layout(width=800, height=300, font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=10, r=10, t=10, b=10))


# make a gaussian i can move
p = m[1].function(xm) * scale
p = p[int(kev2ch(m[1].centre.value-.35)):int(kev2ch(m[1].centre.value+.35))]
x_p = xm[int(kev2ch(m[1].centre.value-.35)):int(kev2ch(m[1].centre.value+.35))]

x_list = []
y_list = []


def add_peak(scale_y, shift_x, scale_x=0, p=p, x_p=x_p, x_list=x_list, y_list=y_list, scale=scale, m=m):
    # x_list.append( (x_p  + shift_x))
    points = len(x_p)
    new_X = np.linspace(x_p[0]-scale_x, x_p[-1]+scale_x, points)
    x_list.append(new_X - x_p[0] -0.35 + shift_x)
    y_list.append( p * scale_y + m[0].function(x_list[-1]) * scale)

# Si Ka
# add_peak(.3, 1.74)

# # add Cu Ka peak
add_peak(.8, 8.0)

# # add cu Kb peak
add_peak(.1, 8.7)

# add Mo Ka at 17.48
add_peak(.2, 17.48, scale_x=0.8)
# add Mo La at 2.29 and Mo Lb at 2.4
add_peak(.22, 2.29, scale_x=-0.2)
# add_peak(.2, 2.4)


# add s
fig.add_trace(go.Scatter(x=x, y=s.data, name='Expected spectrum', line=dict(width=3, color=colors[0])))
for i in range(len(x_list)):
    if i == 0:
        name = True
    else:
        name = False
    fig.add_trace(go.Scatter(x=x_list[i], y=y_list[i], name='Additional stray peaks', line=dict(width=3, color=colors[2]), showlegend=name))
# fig.add_trace(go.Scatter(x=x_si, y=si, name='Si', line=dict(width=3, color=colors[1])))

# legend in upper right corner
fig.update_layout(legend=dict(x=0.99, y=.99, yanchor="top", xanchor="right"))

# fig.update_yaxes(type="log")
fig.update_xaxes(title='Energy [keV]', range=[.39,20])
fig.update_yaxes(title='Intensity [counts]', range=[-1e3,12e3])

In [ ]:
# fig.write_image('../figures/pp_strays.pdf')
# fig.write_image('../../mastersthesis/figures/pp_strays.pdf')

# Cu-tape, showing bg ratio and Fiori (NOT USED)

In [ ]:
def vacc(s):
    return s.metadata.Acquisition_instrument.SEM.beam_energy

In [ ]:
path = "../../../Masteroppgave/2022-09-06_EDS-Apreo/exports/"
s1 = hs.load(path + 'Cu_05kV.emsa', signal_type='EDS_SEM')
s2 = hs.load(path + 'Cu_10kV.emsa', signal_type='EDS_SEM')
s3 = hs.load(path + 'Cu_30kV.emsa', signal_type='EDS_SEM')
ss = [s1, s2, s3]
# s = hs.stack([s1, s2, s3])
elements = ['C', 'Cu', 'O']

scale = s1.axes_manager[0].scale
offset = s1.axes_manager[0].offset
x = s1.axes_manager[0].axis

In [ ]:
mm = []

for s in ss:
    s.set_elements(elements)
    s.data[:33] = 0
    m = s.isig[.13:vacc(s)].create_model()
    m.fit()
    m.fit_background()
    mm.append(m)


In [ ]:
fig = go.Figure() 
fig.update_layout(width=800, height=300, font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=10, r=10, t=10, b=10))


# add s
for s in ss:
    fig.add_trace(go.Scatter(x=x, y=s.data, name=f'{vacc(s)} kV', mode = 'lines'))

for m in mm:
    xm = m.as_signal().axes_manager[0].axis
    b = m[0].function(xm) * scale
    # fig.add_trace(go.Scatter(x=xm, y=b, name='Background', line=dict(width=3, color=colors[0])))
    fig.add_trace(go.Scatter(x=xm, y=m.as_signal().data, name='m', line=dict(width=3)))

# legend in upper right corner
fig.update_layout(legend=dict(x=0.99, y=.99, yanchor="top", xanchor="right"))

# fig.update_yaxes(type="log")
fig.update_xaxes(title='Energy [keV]', range=[.13,2])
fig.update_yaxes(title='Intensity [counts]', range=[.5,5], type='log')

In [ ]:
bg = [42, 12, 125]

In [ ]:
mm[2][1].function(x).sum() * scale

In [ ]:
for s in ss:
    m = mm[ss.index(s)]
    c_ka = m['C_Ka'].A.value
    i_s  =  s.isig[0.18:0.38].data.sum()
    sum_s = s.data.sum()
    bg_c_ka = m[0].function(.277) * scale
    # fiori = round(i_s/bg[ss.index(s)])
    fiori = round(i_s/(bg_c_ka))
    print(f'P/B {vacc(s)} kV: {fiori}')

    # background portions
    lines = 

In [ ]:
for l in mm[2].active_components:
    if not l.isbackground:
        print(l.name, l.A.value, l.centre.value, l.sigma.value)

In [ ]:
peaks = [0,0,0]
lines = [['C_Ka'], ['C_Ka'], ['C_Ka', 'Cu_Ka', 'Cu_Kb']]
for m in mm:
    for l in lines[mm.index(m)]:
        peaks[mm.index(m)] += m[l].A.value


In [ ]:
peaks

In [ ]:
bg_total = [0,0,0]
for m in mm:
    mx = m.as_signal().axes_manager[0].axis
    bg_total[mm.index(m)] = m[0].function(mx).sum() * scale

bg_total

In [ ]:
print('S sum,      m_total,      diff')
for i in range(3):
    print(f'{ss[i].data.sum():.2f},  {peaks[i]+bg_total[i]:.2f}, {ss[i].data.sum() - (peaks[i]+bg_total[i]):.2f}')

In [ ]:
# print background ratio
print('Background ratio')
for i in range(3):
    print(f'{bg_total[i]/(bg_total[i] + peaks[i]):.2f}')